# The best places for placing a supermarket.
**Applied Data Science Capstone by IBM/Coursera**

## Table of contents
* [Introduction](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## Introduction <a name="introduction"></a>

Let's assume that our stakeholder is a supermarket chain. The company operates in a very competitive and volatile environment, so this company has to find new places to expand and leave unpromising locations. This company wants to expand its network in Yekaterinburg city in Russia. But the market in this city is very competitive and we need to do an analysis to find the best places to place supermarkets. Ideally, we need to find crowded places in residential areas without supermarket competitors in the areas.

## Data <a name="data"></a>

We need to find places without supermarkets in the area, with public places nearby like a shopping mall or a business center. Residential areas can be distinguished by the presence of schools, kindergartens, and hospitals (social venues) in a neighborhood. All the venues we need can be found using the Foursquare API. 
Yekaterinburg city is divided not into neighborhoods, but huge districts and it's useless to find places for supermarkets by districts.
I will divide the city by post offices because post offices are evenly distributed throughout the city.
Also, almost every post office has its postal code.

**Official districts of Yekaterinburg**
<img src="https://upload.wikimedia.org/wikipedia/commons/3/34/Ekb_all_districts.svg" width="300px">

### Getting post offices
We need to get post office locations and their postal codes. We can find it with the Foursquare API.
After that, we can see, that Foursquare API returns some locations without postal code. There are also duplicate postal codes. We have to delete such venues.

In [1]:
# @hidden_cell
import requests as rq
import numpy as np
import pandas as pd
import json
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

CLIENT_ID = 'private info'
CLIENT_SECRET = 'private info'
VERSION = '20180604'
RADIUS = 7000
LIMIT = 1000
CATEGORYID = '4bf58dd8d48988d172941735' # post office

lat = '56.8519'
lng = '60.6122'
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        CATEGORYID,
        RADIUS,
        LIMIT)

res = rq.get(url)
post_offices_raw = json.loads(res.text)

In [2]:
post_offices_raw

{'meta': {'code': 200, 'requestId': '5dcfd3100de0d9001cfc9f06'},
 'response': {'venues': [{'id': '4f549edae4b0754d59f8a576',
    'name': 'Почта 620027',
    'location': {'address': 'ул. Мельковская, 2Б',
     'lat': 56.85002408076234,
     'lng': 60.6022097479635,
     'labeledLatLngs': [{'label': 'display',
       'lat': 56.85002408076234,
       'lng': 60.6022097479635}],
     'distance': 642,
     'postalCode': '620027',
     'cc': 'RU',
     'city': 'Екатеринбург',
     'state': 'Свердловская обл.',
     'country': 'Россия',
     'formattedAddress': ['ул. Мельковская, 2Б',
      '620027, Екатеринбург',
      'Россия']},
    'categories': [{'id': '4bf58dd8d48988d172941735',
      'name': 'Post Office',
      'pluralName': 'Post Offices',
      'shortName': 'Post Office',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/postoffice_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1573901074',
    'hasPerk': False},
   {'id': '4cb04504

In [3]:
post_offices_coords = [(row['location']['address'] if 'address' in row['location'] else np.NaN, row['location']['postalCode'] if 'postalCode' in row['location'] else np.NaN, row['location']['lat'], row['location']['lng']) for row in post_offices_raw['response']['venues']]
post_offices_coords_df = pd.DataFrame(post_offices_coords, columns=['Address', 'Postal code', 'Lat', 'Lng'])

In [4]:
post_offices_coords_df.shape

(50, 4)

In [5]:
post_offices_coords_df.dropna(inplace=True)
post_offices_coords_df.shape

(29, 4)

In [6]:
post_offices_coords_df.drop_duplicates(['Postal code'], inplace=True)
post_offices_coords_df.shape

(22, 4)

In [7]:
post_offices_map = folium.Map(location=[lat, lng], zoom_start=10)

for p_code, latitude, longitude in zip(post_offices_coords_df['Postal code'], post_offices_coords_df['Lat'], post_offices_coords_df['Lng']):
    label = folium.Popup(str(p_code))
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        fill=True,
        popup=label).add_to(post_offices_map)

post_offices_map

### Getting venues nearby post offices
The idea is to find places from different categories that can influence the success of the supermarket in a particular place.
We will find and visualize the number of venues in around post offices in every category:
* Social venues
 * Schools
 * Hospitals
* Competitor venues
 * Big Box Stores
 * Supermarkets
 * Food & Drink Shops
* Crowded venues
 * Business Centers
 * Offices
 * Entertainment Centers

As we rely on people coming to a supermarket it must be accessible by walking no more than 15-20 minutes from any place in the area. Assuming the average speed of a person is 6 km/h it will result in a radius of 800 meters.

In [8]:
# it seems that the radius of 800m is allowable for walking
RADIUS = 800
def get_stat_by_categories(post_offices_df, venue_ids, column_name):
    venues = {}
    for item in post_offices_coords_df.index:
        venues[item] = getNearbyVenuesByCategories(post_offices_df.loc[item]['Lat'],
                                                        post_offices_df.loc[item]['Lng'],
                                                        venue_ids,
                                                        RADIUS)
    post_offices_df[column_name] = post_offices_df.index.to_series().apply(lambda x: len(venues[x]))
    return post_offices_df

def print_map(venues_number_df, column_name):
    post_offices_map = folium.Map(location=[lat, lng], zoom_start=10)

    for level, p_code, latitude, longitude in zip(venues_number_df[column_name], venues_number_df['Postal code'], venues_number_df['Lat'], venues_number_df['Lng']):
        label = folium.Popup(str(p_code) + ' ' + str(level))
        folium.CircleMarker(
            [latitude, longitude],
            fill=True,
            radius=level,
            popup=label).add_to(post_offices_map)
    return post_offices_map

def getNearbyVenuesByCategories(latitude, longitude, categories, radius=500):
    
    # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        latitude, 
        longitude, 
        radius, 
        LIMIT,
        ','.join(categories))
    # make the GET request
    results = rq.get(url).json()["response"]['groups'][0]['items']

    # return only relevant information for each nearby venue
    venues_list = [(v['venue']['name'], v['venue']['categories'][0]['name']) for v in results]

    return(venues_list)

**Social venues**

In [9]:
post_offices_coords_df = get_stat_by_categories(post_offices_coords_df, ['4bf58dd8d48988d13b941735', '4bf58dd8d48988d196941735'], 'Social venues')
print_map(post_offices_coords_df, 'Social venues')

**Competitor venues**

In [10]:
post_offices_coords_df = get_stat_by_categories(post_offices_coords_df, ['52f2ab2ebcbc57f1066b8b42', '52f2ab2ebcbc57f1066b8b46', '4bf58dd8d48988d1f9941735'], 'Competitor venues')
print_map(post_offices_coords_df, 'Competitor venues')

**Crowded venues**

In [11]:
post_offices_coords_df = get_stat_by_categories(post_offices_coords_df, ['56aa371be4b08b9a8d573517', '4bf58dd8d48988d124941735', '4bf58dd8d48988d1e1931735'], 'Crowded venues')
print_map(post_offices_coords_df, 'Crowded venues')

**The summary table of the number of venues in each category by postal office**

In [12]:
post_offices_coords_df

,Address,Postal code,Lat,Lng,Social venues,Competitor venues,Crowded venues
0,"ул. Мельковская, 2Б",620027,56.850024,60.602210,10,9,22
1,"ул. Индустрии, 121",620098,56.906339,60.610389,7,17,6
2,"ул. Онежская, 12",620089,56.799071,60.640526,5,9,3
3,"ул. Индустрии, 26а",620042,56.894920,60.593025,13,18,9
4,"ул. Пальмиро Тольятти, 15",620102,56.821634,60.579925,8,11,24
5,ул. Старых Большевиков 75,620135,56.897600,60.622856,11,18,9
6,"Тбилисский бул., 3",620073,56.793290,60.628674,8,16,8
7,"ул. Сыромолотова, 28а/1",620072,56.834896,60.688185,8,13,11
9,"Авиационная ул., 69",620130,56.803521,60.619687,10,18,8
11,"ул. Сурикова, 28",620144,56.812582,60.599535,12,17,19


## Methodology <a name="methodology"></a>
We collected post office coordinates and their postal codes for dividing the city on neighborhoods because it is impossible to analyze with administrative districts because of their huge sizes. After that, we collected data about venues nearby every post office. 
We used venue types of three relevant for us categories: social venues, competitor venues(other supermarket chains) and crowded venues.
Now we need to analyze the data. As I think, the best approach is ranking areas by computing a grade for every area. 
The formula for ranking will be:

$r = soc + crw + 2*cmp$  
$r$ - rank  
$crw$ - number of crowded places  
$cmp$ - number of competitor venues  

The presence of social and crowded venues is a positive factor, the presence of competitor venues is a huge negative factor, so the multiplier is valid in this case.  
Also, we will try to separate the data using the DBSCAN clustering model, because this model is robust to outliers and can find arbitrarily shaped clusters. I want to try to prove ranking results by clustering results.

## Analysis <a name="analysis"></a>
Let's find out the best places for placing a supermarket. So we need to find the grade for every postal office. Also, we will try to find the difference between areas with the clusterization model.

**Grading neighborhoods**

In [13]:
def get_grade(soc_venues_n, comp_venues_n, crowd_venues_n):
    return soc_venues_n + crowd_venues_n - 2*comp_venues_n

graded_post_offices = post_offices_coords_df.copy()
graded_post_offices['Grade'] = graded_post_offices.apply (lambda row: get_grade(row['Social venues'], row['Competitor venues'], row['Crowded venues']), axis=1)

In [14]:
graded_post_offices.sort_values(by='Grade', ascending=False, inplace=True)
graded_post_offices

,Address,Postal code,Lat,Lng,Social venues,Competitor venues,Crowded venues,Grade
28,"просп. Ленина, 39",620000,56.839337,60.608463,13,14,30,15
0,"ул. Мельковская, 2Б",620027,56.850024,60.602210,10,9,22,14
4,"ул. Пальмиро Тольятти, 15",620102,56.821634,60.579925,8,11,24,10
49,"Машиностроителей, 67А",620039,56.882845,60.568017,4,4,9,5
45,"ул. Техническая, д. 62",620050,56.870556,60.522845,7,4,5,4
21,"ул. Седова, 43",620090,56.870267,60.528544,6,5,8,4
13,"ул. Академика Бардина, 21а",620149,56.803654,60.554837,11,10,7,-2
11,"ул. Сурикова, 28",620144,56.812582,60.599535,12,17,19,-3
17,ул.Фрунзе 20,620142,56.815609,60.617630,14,18,19,-3
16,"ул. Восточная, 21",620100,56.827179,60.637856,16,15,9,-5


In [15]:
print_map(graded_post_offices, 'Grade')

**the DBSCAN model as an alternative approach to prove the result.**
Use search grid to find the best params.

In [16]:
df_for_clustering = post_offices_coords_df.loc[:, post_offices_coords_df.columns != 'Address']
best_score = {'eps': 0, 'score': 0}
eps = range(5, 30)
for n in eps:
    kmeans = DBSCAN(eps=n, min_samples=2)
    labels = kmeans.fit(df_for_clustering)
    silhouette_avg = silhouette_score(df_for_clustering, labels.labels_)
    if silhouette_avg > best_score['score']: best_score = {'eps': n, 'score': silhouette_avg}
    print('eps: {}, score: {}'.format(n, silhouette_avg))
print('the best clusters number: {}'.format(best_score['eps']))

eps: 5, score: -0.10256791769146698
eps: 6, score: -0.03682754190523516
eps: 7, score: 0.02042905128481953
eps: 8, score: 0.02042905128481953
eps: 9, score: 0.25039235024335765
eps: 10, score: 0.36108044836156883
eps: 11, score: 0.36108044836156883
eps: 12, score: 0.28892517676926077
eps: 13, score: 0.4094089593550984
eps: 14, score: 0.4094089593550984
eps: 15, score: 0.4430671932074194
eps: 16, score: 0.4430671932074194
eps: 17, score: 0.4810417502408585
eps: 18, score: 0.5567432733646123
eps: 19, score: 0.5022568208932578
eps: 20, score: 0.46278608565523566
eps: 21, score: 0.46278608565523566
eps: 22, score: 0.46278608565523566
eps: 23, score: 0.46278608565523566
eps: 24, score: 0.46278608565523566
eps: 25, score: 0.46278608565523566
eps: 26, score: 0.46278608565523566
eps: 27, score: 0.46278608565523566
eps: 28, score: 0.46278608565523566
eps: 29, score: 0.5525207672042175
the best clusters number: 18


In [17]:
clustering = DBSCAN(eps=best_score['eps'], min_samples=2).fit(df_for_clustering)
df_for_clustering.insert(0, 'Cluster Labels', clustering.labels_)

In [18]:
df_for_clustering.head()

,Cluster Labels,Postal code,Lat,Lng,Social venues,Competitor venues,Crowded venues
0,-1,620027,56.850024,60.602210,10,9,22
1,0,620098,56.906339,60.610389,7,17,6
2,0,620089,56.799071,60.640526,5,9,3
3,1,620042,56.894920,60.593025,13,18,9
4,0,620102,56.821634,60.579925,8,11,24


In [19]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[lat, lng], zoom_start=11)
x = np.arange(len(set(clustering.labels_)))
ys = [i + x + (i*x)**2 for i in range(len(set(clustering.labels_)))]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(df_for_clustering['Lat'], df_for_clustering['Lng'], df_for_clustering['Postal code'], df_for_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
map_clusters

## Results and Discussion <a name="results"></a>
As we can see, the best results by the ranking are highlighted by DBSCAN as outliers, It means that these results are different from other neighborhoods.
So the DBSCAN approach almost completely proved the ranking result.

Let's look the best postal codes:
* Ranking:
 * 620000
 * 620027
 * 620102
* Clustering:
 * 620000
 * 620027
 * 620012

620102 and 620012 areas have not such a good grade as 620000 and 620000, so let's exclude these areas.
So the best places for placing a supermarket are around these post offices:

In [20]:
result_df = post_offices_coords_df[post_offices_coords_df['Postal code'].isin(['620000', '620027'])]
result_df

,Address,Postal code,Lat,Lng,Social venues,Competitor venues,Crowded venues
0,"ул. Мельковская, 2Б",620027,56.850024,60.602210,10,9,22
28,"просп. Ленина, 39",620000,56.839337,60.608463,13,14,30


The result is like the truth. These places are residential areas with crowded places in the proximity. 
We considered the vicinity of 800 meters around every coordinate, so stakeholders should find places in a radius 800 meters from the post offices.
However, the result isn't ideal, because we considered only venues nearby post offices and we could skip some suitable places for supermarkets. Also, we relied on the most popular venue types and didn't use rare types of venues.

## Conclusion <a name="conclusion"></a>
The purpose of this project is to find the best places to place supermarkets in the city with a competitive market.
We divided the city into neighborhoods by postal offices and found relevant venues with Foursquare API.
After the analysis, we have found two of the best places for placing a supermarket.
The final decision on optimal supermarket location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like availability of retail space or district welfare level. 